In [ ]:
!pip install -U spacy
#first install the library that would help us use BERT in an easy to use interface
#https://github.com/UKPLab/sentence-transformers/tree/master/sentence_transformers
!pip install -U sentence-transformers

     |████████████████████████████████| 6.0 MB 3.8 MB/s 
     |████████████████████████████████| 10.1 MB 42.4 MB/s 
     |████████████████████████████████| 181 kB 68.6 MB/s 
     |████████████████████████████████| 451 kB 58.3 MB/s 
     |████████████████████████████████| 628 kB 26.5 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 78 kB 

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 99 kB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
## Use this to import spacy directly if using the Brain colab runtime or a custom colab runtime that includes spacy in its build.
import os
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy

In [ ]:
text = """I am looking for a sushi restaurant in Tenochtitlán that has more than 20 items on the menu and has dishes with less than $10. It should be open all day."""
doc = nlp(text)
sentence_spans = list(doc.sents)
displacy.render(doc, jupyter = True, style="ent")

In [ ]:
stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl

embedder = SentenceTransformer('all-MiniLM-L6-v2')
#embedder = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv("https://storage.googleapis.com/maba-class-data-6490/combined_paris.csv")

In [ ]:
df['Hotel'].drop_duplicates()

0                   Le Relais des Halles
30                Hotel George - Astotel
60      Hotel Bradford Elysees - Astotel
90                   Hotel 34B - Astotel
120         Hotel la Nouvelle Republique
                      ...               
2709                     Esmeralda Hotel
2739           Hotel Royal Saint Germain
2769                         Hotel Artus
2799               Hotel Marais Bastille
2829                      Hotel Brittany
Name: Hotel, Length: 94, dtype: object

In [ ]:
df_combined = df.sort_values(['Hotel']).groupby('Hotel', sort=False).review.apply(''.join).reset_index(name='all_review')
  

In [ ]:
df_combined

,Hotel,all_review
0,25hours Hotel Terminus Nord,We’ve spent lots of time in Paris and this was...
1,Acacias Etoile Hotel,The hotel is great for value. The breakfast se...
2,COQ Hotel Paris,Stayed for a short city break. The hotel is a...
3,Campanile Paris 14 - Maine Montparnasse,Room was very clean & transportation is very n...
4,Cler Hotel,We had the BEST stay at Cler Hotel. The locat...
...,...,...
89,Sofitel Paris Le Faubourg,"4 years ago, I was the last time at Sofitel Le..."
90,St Christopher's Gare du Nord Paris,"When arriving to the area, it felt a little da..."
91,St Christopher's Inn Canal Paris,I’ve stayed at St Christopher Inn Canal in Par...
92,Touring Hotel,Hotel is in a great location - minutes walk fr...


In [ ]:
import re

df_combined['all_review'] = df_combined['all_review'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['all_review']= df_combined['all_review'].apply(lambda x: lower_case(x))


In [ ]:
df = df_combined

In [ ]:
df_sentences = df_combined.set_index("all_review")

In [ ]:
df_sentences.head()

In [ ]:
df_sentences = df_sentences["Hotel"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

94

In [ ]:
list(df_sentences.keys())[:5]

['weve spent lots of time in paris and this was a quick trip such a fun cozy welcoming spot we wish wed had more time there our room was palatial by paris standards and very quiet and comfortable no complaintscentrally locates and new hotel very good breakfast and room quality centrally location just outside gare du nord and the metro friendly staff with nice bar and insite restaurant and within easy walking distance to other interesting areas in paristhe hotel is pretty but the service standard seem to have been established by an 18 years old trainee on the day of our checkout the housekeeping staff knocked and entered the room 5 times successively starting at 9am all the way until 1230pm very shortterm memory this goes against hotel standards whereby hsk is informed of the room vacancy by the front office once the guest has checked out  on a side note the elevator is extremely slow and antique  highly recommend giving up and using the stairs instead  also in 2021 in covid era there a

In [ ]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [ ]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]


100%|██████████| 94/94 [00:00<00:00, 301656.14it/s]


In [ ]:
# Corpus with example sentences
corpus = df_sentences_list
corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
paraphrases = util.paraphrase_mining(model, corpus)
query_embeddings_p =  util.paraphrase_mining(model, queries,show_progress_bar=True)

NameError: ignored

In [ ]:
# import pickle as pkl
# with open("/content/drive/MyDrive/BertSentenceSimilarity/Pickles/corpus_embeddings.pkl" , "wb") as file_:
#  pkl.dump(corpus_embeddings,file_)

In [ ]:
queries = ['Hotel closest to eiffel tower',
           'hotel closest to Gare du Nord'
           ]
query_embeddings = embedder.encode(queries,show_progress_bar=True)


In [ ]:

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df['all_review']== corpus[idx]]
        print("paper_id:  " , row_dict['Hotel'] , "\n")
        # print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        # print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        # print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")


Top 5 most similar sentences in corpus:


==========================Query==============================
=== Hotel closest to eiffel tower =====
Score:    (Score: 0.6756) 

Paragraph:    the hotel is placed on a very beautiful street with a view of the eiffel tower the rooms are pleasant the reception works 247 which is crucial for us since we are walking till late night the cleaning services is amazing once a dayeverything was great the room the staff the food the location my favourite part was the fact we could see the eiffel tower from our room and it was only a 10min walk the staff are so nice and the breakfast is greatthe locations is very good close to the metro many shops along the street and nice restaurants clean rooms and newly furbished bathrooms windows are soundproof staff is nice and friendly only minus would be that breakfast is served at the table and every morning same choice but you can order more so it is big enough my room will need very soon a carpet to be renewed 

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
embedder = SentenceTransformer('all-MiniLM-L6-v2')

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [ ]:



# Query sentences:
queries = ['Hotel closest to eiffel tower',
           'walking distance to Gare du Nord'
           ]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("(Score: {:.4f})".format(score))
        print(corpus[idx], "(Score: {:.4f})".format(score))
        row_dict = df.loc[df['all_review']== corpus[idx]]
        print("paper_id:  " , row_dict['Hotel'] , "\n")
    # for idx, distance in results[0:closest_n]:
    #     print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
    #     print("Paragraph:   ", corpus[idx].strip(), "\n" )
    #     row_dict = df.loc[df['all_review']== corpus[idx]]
    #     print("paper_id:  " , row_dict['Hotel'] , "\n")
    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """






Query: Hotel closest to eiffel tower

Top 5 most similar sentences in corpus:
(Score: 0.6756)
the hotel is placed on a very beautiful street with a view of the eiffel tower the rooms are pleasant the reception works 247 which is crucial for us since we are walking till late night the cleaning services is amazing once a dayeverything was great the room the staff the food the location my favourite part was the fact we could see the eiffel tower from our room and it was only a 10min walk the staff are so nice and the breakfast is greatthe locations is very good close to the metro many shops along the street and nice restaurants clean rooms and newly furbished bathrooms windows are soundproof staff is nice and friendly only minus would be that breakfast is served at the table and every morning same choice but you can order more so it is big enough my room will need very soon a carpet to be renewed however rooms are quite big bigger than in many hotels in paris i would come back on my n

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
embeddings = model.encode(corpus)
#print(embeddings)

In [ ]:
# Query sentences:
queries = ['Hotel closest to eiffel tower',
           'outside Gare du Nord'
           ]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("(Score: {:.4f})".format(score))
        print(corpus[idx], "(Score: {:.4f})".format(score))
        row_dict = df.loc[df['all_review']== corpus[idx]]
        print("paper_id:  " , row_dict['Hotel'] , "\n")





Query: Hotel closest to eiffel tower

Top 5 most similar sentences in corpus:
(Score: 0.5234)
hotel was very clean and close to the eiffel tower as you can walk lots of good restaurants on the street as well room was not big but very nice breakfast was okay i would recommend going to nearby restaurants definitely recommendhotel was wonderful the room we stayed in was small but enough for two people there are a lot of restaurants and cafes around that you can walk to and have different flavors of food from all over they clean your room everyday if requested i would recommend this hotelwe stayed at the hotel bosquet the second week in august  the rooms are larger than most small parisian hotels and very comfortable  the hotel has a very relaxed friendly atmosphere   this will be the special place that i always come back to when in paris  the location is perfect on a quiet side street with rue cler steps away on one side and the wonderful avenue de la bourdonnais on the other side  th

In [ ]:
df

,Hotel,all_review
0,25hours Hotel Terminus Nord,weve spent lots of time in paris and this was ...
1,Acacias Etoile Hotel,the hotel is great for value the breakfast sel...
2,COQ Hotel Paris,stayed for a short city break the hotel is a ...
3,Campanile Paris 14 - Maine Montparnasse,room was very clean transportation is very ne...
4,Cler Hotel,we had the best stay at cler hotel the locati...
...,...,...
89,Sofitel Paris Le Faubourg,4 years ago i was the last time at sofitel le ...
90,St Christopher's Gare du Nord Paris,when arriving to the area it felt a little dan...
91,St Christopher's Inn Canal Paris,ive stayed at st christopher inn canal in pari...
92,Touring Hotel,hotel is in a great location minutes walk fro...


In [ ]:
hits = util.semantic_search(query_embedding, embeddings, top_k=5)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
  print (hit)
  print("(Score: {:.4f})".format(hit['score']))
  print(corpus[hit['corpus_id']])
  row_dict = df.loc[df['all_review']== corpus[hit['corpus_id']]]
  print("paper_id:  " , row_dict['Hotel'] , "\n")

{'corpus_id': 77, 'score': 0.24323494732379913}
(Score: 0.2432)
the hotel is on a small beautiful unknown square right next to the cnam on the right bank close to grand boulevard st martin areas and super accessible from the gare du nord or montparnasse via the line 4   ive stayed here several times and always had a really nice view of the square   the staff is very welcoming and competentthe little palace hotel as its name suggests is small the rooms are quite tiny and there isnt much if any closet space but it wonderfully located and a beautiful spot if youre looking for a grand hotel this isnt for you but if you are looking for a nice little room to relax in when youre not walking the city i highly recommend this hotel i dont think it would be large enough for two people but for one it is enough room the staff was very helpful and kind and its minutes from wonderful shops and cafes i will definitely come back my personal favorite was the small terrace where i took my coffee in the m

In [ ]:
max_frequency=max(word_frequencies.values())
for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency

In [ ]:
sentence_tokens= [sent for sent in doc.sents]
print(sentence_tokens)

[Came with friends for a small break in Paris and we had a great stay as the hotel's location is very close to Arc de Triomphe and Champs Elysees., Breakfast is good and Staff are really nice., Just an advice, don't book the room on the ground floor as it is very tiny and you cannot move or have more than a bag., Reception staff was exceedingly cooperative with room changes and dinner bookings., Air conditioning could be better as airflow/circulation is insufficient., Please note that lift is VERY small (3 pax maximum)We were 5 persons in a business trip., Excellent location for us in this trip and good help from the hotel to find good restaurants and to help in general., Had good food all days., Not their fault how difficult it is to get Taxis at rush hours in Paris., Well situated clean and nice even though the rooms are small like they are so often in Paris., Simple but fine breakfast., My only negative comments is that they served cheese in small doses wrapped in plastic at breakfa

In [ ]:
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():                            
             sentence_scores[sent]=word_frequencies[word.text.lower()]
            else:
             sentence_scores[sent]+=word_frequencies[word.text.lower()]

In [ ]:
from heapq import nlargest
select_length=int(len(sentence_tokens)*0.3)
select_length
summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
summary

[The location is excellent for central Paris, the rooms were a good size, and well presented (and clean), the staff - friendly and helpful.,
 Excellent location for us in this trip and good help from the hotel to find good restaurants and to help in general.,
 Came with friends for a small break in Paris and we had a great stay as the hotel's location is very close to Arc de Triomphe and Champs Elysees.,
 Location is great nice, neighborhood near Arc de Triomphe has many nice restaurants and cafes.,
 A short stroll from every thing you need to do…metro line…bistro…bars….Arc de Triomphe.,
 Well situated clean and nice even though the rooms are small like they are so often in Paris.,
 The hotel was very clean and the rooms were very nice.,
 Room was compact (around 10' by 12') and lift up small (can only take one person and no wheelchairs), but otherwise clean, modern and comfortable.,
 Metro line 1 nearby, many bistros and restaurants to chose from, internet cafe on Avenue Carnot Access

In [ ]:
final_summary=[word.text for word in summary]
final_summary
summary=''.join(final_summary)
summary

"The location is excellent for central Paris, the rooms were a good size, and well presented (and clean), the staff - friendly and helpful.Excellent location for us in this trip and good help from the hotel to find good restaurants and to help in general.Came with friends for a small break in Paris and we had a great stay as the hotel's location is very close to Arc de Triomphe and Champs Elysees.Location is great nice, neighborhood near Arc de Triomphe has many nice restaurants and cafes.A short stroll from every thing you need to do…metro line…bistro…bars….Arc de Triomphe.Well situated clean and nice even though the rooms are small like they are so often in Paris.The hotel was very clean and the rooms were very nice.Room was compact (around 10' by 12') and lift up small (can only take one person and no wheelchairs), but otherwise clean, modern and comfortable.Metro line 1 nearby, many bistros and restaurants to chose from, internet cafe on Avenue Carnot Access to the Arc de Triomphe,

In [ ]:
doc = df_combined['all_review'][0]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + doc, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0]))

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

<pad> the hotel is pretty but the service standard seem to have been established by an 18 year old trainee on the day of our checkout the housekeeping staff knocked and entered the room 5 times successively starting at 9am all the way until 1230pm very shortterm memory this goes against hotel standards. the elevator is extremely slow and antique highly recommend giving up and using the stairs instead of using the stairs.</s>


In [ ]:
from transformers import pipeline

# Open and read the article
#f = open("article.txt", "r", encoding="utf8")
to_tokenize = doc

# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization",max_length = 500)
summarized = summarizer(to_tokenize, min_length=25, max_length=50)

# Print summarized text
print(summarized)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)
